In [1]:
import whisper
import torch
import numpy as np
import queue
import time
from pydub import AudioSegment

# ⚡ Выбираем устройство (GPU или CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎯 Используем устройство: {device}")

# ⚙️ Загружаем модель Whisper
model = whisper.load_model("medium", device=device)  # Можно "small" или "large"

# 🎵 Загружаем MP3-файл
AUDIO_FILE = "example.mp3"
audio = AudioSegment.from_mp3(AUDIO_FILE).set_frame_rate(16000).set_channels(1)

# 📌 Конвертируем в numpy
samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0  # Нормализация

# 🎤 Параметры обработки
SAMPLERATE = 16000
CHUNK_DURATION = 5  # Длина чанка в секундах
OVERLAP = 2         # Перекрытие для плавности
CHUNK_SIZE = CHUNK_DURATION * SAMPLERATE
OVERLAP_SIZE = OVERLAP * SAMPLERATE
QUEUE = queue.Queue()

# 🔄 Буфер аудио
audio_buffer = np.zeros(CHUNK_SIZE + OVERLAP_SIZE, dtype="float32")
last_text = ""

# 🔄 Симуляция потока
def stream_audio(data, chunk_size):
    for start in range(0, len(data), chunk_size):
        chunk = data[start:start + chunk_size]
        if len(chunk) < chunk_size:
            break  # Конец аудио
        QUEUE.put(chunk)
        time.sleep(chunk_size / SAMPLERATE)  # Имитация реального времени

# 🚀 Запуск потока
print("🔄 Стриминг MP3...")
stream_audio(samples, CHUNK_SIZE)

# 🔍 Обработка чанков
while not QUEUE.empty():
    new_data = QUEUE.get()

    # 🔄 Сдвиг буфера и добавление новых данных
    audio_buffer[:-len(new_data)] = audio_buffer[len(new_data):]
    audio_buffer[-len(new_data):] = new_data

    # 🎙️ Транскрибируем
    print("🔍 Распознаём...")
    result = model.transcribe(audio_buffer, language="ru")

    # 📜 Получаем текст
    current_text = result["text"].strip()

    # 🔥 Вывод финального текста
    print(f"📜 Распознанный текст: {current_text}")
    print("=" * 50)

print("✅ Завершено!")


ModuleNotFoundError: No module named 'sounddevice'